<a href="https://colab.research.google.com/github/Anirudh1905/Fake-News-Classifier/blob/master/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Classifier

## Importing the Data

In [13]:
import pandas as pd

In [14]:
df=pd.read_csv('/content/drive/My Drive/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df.dropna(inplace=True)

In [16]:
x=df.drop('label',axis=1)
y=df['label']

In [17]:
x.shape

(18285, 4)

In [18]:
y.shape

(18285,)

In [57]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [20]:
vocab_size=10000

In [21]:
messages=x.copy()

In [22]:
messages['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [31]:
messages.reset_index(inplace=True)

In [32]:
import nltk 
import re
from nltk.corpus import stopwords

In [33]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Data Preprocessing

In [34]:
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()
corpus=[]
for i in range(0,len(messages)):
  rv=re.sub('[^a-zA-Z]',' ',messages['text'][i])
  rv=rv.lower()
  rv=rv.split()
  rv=[lm.lemmatize(word) for word in rv if not word in stopwords.words('english')]
  rv=' '.join(rv)
  corpus.append(rv)

In [35]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted darrell lucus october subscribe jason chaffetz stump american fork utah image courtesy michael jolley available creative common license apology keith olbermann doubt worst person world week fbi director james comey according house democratic aide look like also know second worst person well turn comey sent infamous letter announcing fbi looking email may related hillary clinton email server ranking democrat relevant committee hear comey found via tweet one republican committee chairman know comey notified republican chairman democratic ranking member house intelligence judiciary oversight committee agency reviewing email recently discovered order see contained classified information long letter went oversight committee chairman jason chaffetz set political world ablaze tweet fbi dir informed fbi learned existence email appear pertinent investigation case reopened jason chaffetz jasoninthehouse october course know case comey a

In [39]:
one_hot_rp=[one_hot(words,vocab_size)for words in corpus]
one_hot_rp[0]

[5836,
 9284,
 6804,
 2141,
 4486,
 6432,
 1813,
 7310,
 536,
 1503,
 5948,
 1604,
 9890,
 419,
 7310,
 536,
 2328,
 3674,
 2261,
 9243,
 7303,
 7176,
 6857,
 4717,
 6309,
 2850,
 3029,
 2305,
 2540,
 3933,
 291,
 910,
 8620,
 4308,
 7316,
 6199,
 4963,
 6768,
 9530,
 6432,
 5652,
 5836,
 1689,
 6804,
 1053,
 79,
 3447,
 3693,
 1323,
 8620,
 4308,
 3190,
 990,
 6432,
 1922,
 371,
 1813,
 3371,
 4963,
 4217,
 117,
 2122,
 3262,
 7599,
 4954,
 117,
 5658,
 3016,
 5873,
 9384,
 6434,
 3008,
 6432,
 3826,
 1017,
 8250,
 62,
 4641,
 6434,
 8433,
 3693,
 6432,
 2478,
 4641,
 8433,
 1689,
 3016,
 4721,
 5836,
 6678,
 4619,
 3158,
 6434,
 8639,
 9535,
 117,
 9363,
 717,
 1026,
 4486,
 5175,
 6682,
 323,
 7046,
 1813,
 2543,
 3158,
 6434,
 8433,
 7310,
 536,
 9783,
 4344,
 7316,
 8215,
 8250,
 4963,
 8261,
 4458,
 4963,
 3980,
 9405,
 117,
 3838,
 8545,
 9606,
 3574,
 8830,
 7310,
 536,
 4984,
 9890,
 3558,
 3693,
 3574,
 6432,
 541,
 2320,
 9535,
 117,
 2683,
 1288,
 3574,
 3693,
 5588,
 8601,

## Embedding Representation

In [41]:
s_len=len(max(corpus))
s_len

524

In [42]:
embedding_docs=pad_sequences(one_hot_rp,padding='pre',maxlen=s_len)
embedding_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0, 5836, 9284, 6804, 2141, 4486, 6432, 1813, 7310,
        536, 1503, 5948, 1604, 9890,  419, 7310,  536, 2328, 3674, 2261,
       9243, 7303, 7176, 6857, 4717, 6309, 2850, 3029, 2305, 2540, 3933,
        291,  910, 8620, 4308, 7316, 6199, 4963, 6768, 9530, 6432, 5652,
       5836, 1689, 6804, 1053,   79, 3447, 3693, 1323, 8620, 4308, 3190,
        990, 6432, 1922,  371, 1813, 3371, 4963, 42

## Creating Model

In [58]:
emb_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,emb_vector_features,input_length=s_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 524, 40)           400000    
_________________________________________________________________
dropout (Dropout)            (None, 524, 40)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [59]:
len(embedding_docs),y.shape

(18285, (18285,))

In [60]:
import numpy as np
x_final=np.array(embedding_docs)
y_final=np.array(y)

In [61]:
x_final.shape,y_final.shape

((18285, 524), (18285,))

In [62]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=33)

## Model Training

In [63]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 40ms/step - loss: 0.4128 - accuracy: 0.8049 - val_loss: 0.2349 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 8s 39ms/step - loss: 0.1528 - accuracy: 0.9469 - val_loss: 0.2018 - val_accuracy: 0.9264
Epoch 3/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0883 - accuracy: 0.9709 - val_loss: 0.2146 - val_accuracy: 0.9302
Epoch 4/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0564 - accuracy: 0.9838 - val_loss: 0.2363 - val_accuracy: 0.9284
Epoch 5/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0426 - accuracy: 0.9884 - val_loss: 0.2455 - val_accuracy: 0.9273
Epoch 6/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0376 - accuracy: 0.9871 - val_loss: 0.3321 - val_accuracy: 0.8689
Epoch 7/10
192/192 [==============================] - 8s 39ms/step - loss: 0.0586 - accuracy: 0.9812 - val_loss: 0.2956 - val_accuracy: 0.9183

## Evaluation of the Model

In [64]:
y_pred=model.predict_classes(x_test)
y_pred[0]

array([1], dtype=int32)

In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3217,  245],
       [ 229, 2344]])

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)*100

92.14581607290803